# Navigation

---

Playground for deep reinforcement learning with Unity ML-Agents environment.

### 1. Start the Environment


In [1]:
from unityagents import UnityEnvironment
import numpy as np

Starting the environment!
Adjust the `file_name` parameter to the location of the downloaded Unity environment (see README).


In [2]:
env = UnityEnvironment(file_name="Banana_Linux/Banana.x86")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Unity-ML environments contain **_brains_** which are responsible for deciding the actions of their associated agents.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 


In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37


### 3. Take Random Actions in the Environment

Performing random actions and getting a reward from the environment till the environment returns done.


In [ ]:
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

### 4. Intelligence incoming!


In [5]:
from dqn_agent import Agent
import torch
from collections import deque

# using a double dueling dqn with noisy linear layers (-> no epsilon needed for training)
agent = Agent(state_size=state_size, action_size=action_size, seed=0, dueling_dqn=True)

def dqn(n_episodes=2000, max_t=1000):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    max_value = 13
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]
        score = 0
        for t in range(max_t):
            # choose action
            action = agent.act(state)
            
            # apply action
            env_info = env.step(action)[brain_name]
            next_state = env_info.vector_observations[0]
            reward = env_info.rewards[0]
            done = env_info.local_done[0]

            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=max_value:
            max_value = np.mean(scores_window)
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
    return scores

scores = dqn()

Use dueling dqn
Episode 100	Average Score: 4.29
Episode 200	Average Score: 10.78
Episode 300	Average Score: 11.50
Episode 400	Average Score: 12.64
Episode 500	Average Score: 12.76
Episode 600	Average Score: 12.30
Episode 700	Average Score: 12.91
Episode 800	Average Score: 13.51
Episode 900	Average Score: 13.08
Episode 1000	Average Score: 13.06
Episode 1100	Average Score: 12.83
Episode 1200	Average Score: 13.49
Episode 1300	Average Score: 14.15
Episode 1400	Average Score: 14.72
Episode 1500	Average Score: 13.82
Episode 1600	Average Score: 12.98
Episode 1700	Average Score: 13.21
Episode 1800	Average Score: 13.83
Episode 1900	Average Score: 13.46
Episode 2000	Average Score: 13.53


In [6]:
from dqn_agent import Agent
import torch
from collections import deque

from utils.nstep_buffer import NStepBuffer

# using a double dueling dqn with noisy linear layers (-> no epsilon needed for training)
agent = Agent(state_size=state_size, action_size=action_size, seed=0, dueling_dqn=True)

# Training with a NStepBuffer
def dqn_with_nstep_buffer(n_episodes=700, max_t=1000):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
    """
    # list containing scores from each episode
    scores = []                        
    # last 100 scores
    scores_window = deque(maxlen=100)  
    step_buffer = NStepBuffer(step_size=2, gamma=0.99)
    max_value = 13
    
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]
        score = 0
        for t in range(max_t):
            # choose action
            action = agent.act(state)
            
            # apply action
            env_info = env.step(action)[brain_name]
            next_state = env_info.vector_observations[0]
            reward = env_info.rewards[0]
            done = env_info.local_done[0]

            step_buffer.append(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 

        for state, action, reward, next_state, done in step_buffer.get_all():
            agent.step(state, action, reward, next_state, done)
        
        # save most recent score
        scores_window.append(score)
        scores.append(score)              
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        average_score = np.mean(scores_window)
        if average_score >= max_value:
            max_value = average_score
            print('\rSaved model with average Score: {:.2f}'.format(average_score))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
    return scores

scores = dqn_with_nstep_buffer()

Use dueling dqn
Episode 100	Average Score: 6.72
Episode 200	Average Score: 9.47
Episode 300	Average Score: 12.67
Saved model with average Score: 13.05
Saved model with average Score: 13.10
Saved model with average Score: 13.15
Saved model with average Score: 13.18
Saved model with average Score: 13.21
Saved model with average Score: 13.21
Saved model with average Score: 13.22
Saved model with average Score: 13.29
Saved model with average Score: 13.32
Saved model with average Score: 13.35
Saved model with average Score: 13.35
Saved model with average Score: 13.45
Saved model with average Score: 13.46
Saved model with average Score: 13.49
Saved model with average Score: 13.59
Saved model with average Score: 13.64
Saved model with average Score: 13.67
Saved model with average Score: 13.80
Saved model with average Score: 13.82
Saved model with average Score: 13.88
Saved model with average Score: 13.90
Saved model with average Score: 13.94
Saved model with average Score: 13.98
Saved model w

In [8]:
env.close()

In [7]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# Data for plotting
episode_numbers = np.arange(1, len(scores)+1, 1)

fig, ax = plt.subplots()
ax.plot(episode_numbers, scores)

ax.set(xlabel='# episode', ylabel='reward',
       title='Training results')
ax.grid()

fig.savefig("training_results.png")
plt.show()

<Figure size 640x480 with 1 Axes>